#  📚 3.5相恋篇-Keras常用接口


         ✅能今天做好的事就不要等到明天。以梦为马，学习趁年华.

----------------------------------------------------------------------------------------------------------------

Keras 是一个用于构建和训练深度学习模型的高阶 API。它可用于快速设计原型、高级研究和生产。 Keras方便用户使用、模块化和可组合、易于扩展。

# 一、本节目标
        本节将讲述Keras 接口，主要目的是可以直接调用我们需要使用的一些API。

# 二、导入keras
  

注意：Keras 与 tf.keras 的区别与联系。
     
     Keras 可以理解为一套搭建与训练神 经网络的高层 API 协议，Keras 本身已经实现了此协议，可以方便的调用 TensorFlow，CNTK 等后端完成加速计算;在 TensorFlow 中，也实现了一套 Keras 协议，即 tf.keras，但只能基于TensorFlow 后端计算，并对 TensorFlow 的支持更好。对于使用 TensorFlow 的开发者来说，tf.keras 可以理解为一个普通的子模块，与其他子模块，如 tf.math，tf.data 等并没有什么差别。下文如无特别说明，Keras 均指代 tf.keras 实现，而不是标准的 Keras实现。

In [1]:
import tensorflow as tf #导入tensorflow
from tensorflow import keras  # 导入keras模型，不能使用import keras，它导入的是标准的Keras库
from tensorflow.keras import layers  ## 导入常见网络层类
print(tf.__version__)  #打印tensorflow版本
print(tf.keras.__version__) #打印keras版本

2.4.1
2.4.0


# 三、常见的功能模块

keras模块里面 有什么，从下图可以看到。

<img src="https://tianchi-public.oss-cn-hangzhou.aliyuncs.com/public/files/forum/161606476184389491616064758850.png"/>

## 3.1 常见的网络层类

    在 tf.keras.layers 命名空间(下文使用 layers 指代 tf.keras.layers)中提供了大量常见网络 层的类接口，如全连接层，激活函数层，池化层，卷积层，循环神经网络层等等。

In [12]:
x = tf.constant([3.,4.,0.1])
layer = layers.Softmax(axis=-1) # 创建Softmax层
layer(x) # 调用softmax前向计算

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.26501969, 0.7203982 , 0.01458223], dtype=float32)>

## 3.2 网络容器

   对于常见的一些网络，需要手动调用每一层的类实例完成前向传播运算，当网络层数变得较深时，这一部分代码显得非常臃肿。可以通过 Keras 提供的网络容器 Sequential 将多个网络层封装成一个大网络模型，只需要调用网络模型的实例一次即可完成数据从第一层到最末层的顺序运算。下面对于三层的神经网络加上单独的 激活层通过 Sequential 容器封装为一个model网络。

In [16]:
from tensorflow.keras import layers, Sequential # 导入Sequential容器
model = tf.keras.models.Sequential([ # 封装为一个网络
    tf.keras.layers.Dense(128, activation='relu'), # 全连接层
    tf.keras.layers.Dense(64, activation='relu'), # 全连接层
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2, activation='softmax')
])
x = tf.random.normal([6,7])

In [17]:
x

<tf.Tensor: shape=(6, 7), dtype=float32, numpy=
array([[ 0.09024575,  0.6943183 ,  1.4205656 , -0.37284467,  0.11625372,
         1.1633333 ,  0.17192082],
       [-1.468165  ,  0.7398397 ,  1.3192202 ,  1.0622461 ,  0.40693176,
         1.0111657 , -0.6690652 ],
       [-0.4337192 ,  0.3768283 ,  1.2944885 , -2.0274334 , -1.8796916 ,
         0.73578995,  0.9881393 ],
       [ 0.855242  ,  0.7737999 , -1.1960135 ,  0.70810556,  1.1527659 ,
         0.58769417,  1.3112597 ],
       [-0.2059029 , -0.5154935 ,  1.4416016 ,  2.0812798 ,  0.34652874,
         0.76385486,  0.41653606],
       [-0.9959193 ,  1.5766927 ,  0.8838324 , -0.7332958 , -0.42769852,
         0.5695647 ,  0.65304685]], dtype=float32)>

In [19]:
model.build(input_shape=(None, 4)) # 创建网络参数
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 128)               640       
_________________________________________________________________
dense_12 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 2)                 130       
Total params: 9,026
Trainable params: 9,026
Non-trainable params: 0
_________________________________________________________________


        当需要在不修改model网络的前提下，想增加网络层数时，可以通过 add()方法继续追加新的网络层，实现动态创建网络的功能:可以看到在原先的网络上新增了一些网络。下面的代码通过指定for循环可以创建对应层数的网络结构，在完成网络创建时，很多类并没有创建内部权值张量等成员变量，此时通过调用类的 build 方法并指定输入大小，即可自动创建所有层的内部张量。通过 summary()函数可以方便打印出网络结构 和参数量:

In [20]:
for _ in range(2):
    model.add(layers.Dense(3)) # 添加全连接层 
    model.add(layers.ReLU())# 添加激活函数层
model.build(input_shape=(None, 4)) # 创建网络参数
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 128)               640       
_________________________________________________________________
dense_12 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 2)                 130       
_________________________________________________________________
dense_14 (Dense)             (None, 3)                 9         
_________________________________________________________________
re_lu_2 (ReLU)               (None, 3)                 0         
_________________________________________________________________
dense_15 (Dense)             (None, 3)                

# 四、网络训练实战

    这里以MNIST数据集为例子作为训练 。在训练网络时，大致的流程是通过前向传播计算网络的预测输出值，然后通过损失函数计算网络预测出的值与真实标签之间的误差，然后通过自动求导工具计算梯度并更新。对于这种常用的训练流程，可以直接通过 Keras 提供的高层接口实现，简洁清晰。

## 4.1 模型设计

在 Keras 中，有 2 个比较特殊的类:keras.Model 和 keras.layers.Layer 类。 Layer类是网络层的母类，定义了网络层的一些常见功能，如添加权值，管理权值列表等。Model 类是网络的母类，除了具有 Layer 类的功能，还添加了保存、加载模型，训练与测 试模型等便捷功能。Sequential 也是 Model 的子类，因此具有 Model 类的所有功能。

    自定义损失函数只需要定义一个函数，并在compile的时候调用该自定义损失函数即可。损失函数有两个默认参数，分别为实际输出、预测输出，如果损失函数只需要这两个参数，那么设计起来就很方便。  
    
    自定义损失函数有两种情况，一种比较简单，只需要默认参数，不需要加入其它参数，而另一种参数稍显复杂，需要其他的一些参数。

In [21]:
import tensorflow as tf #导入tensorflow库
import tensorflow.keras as keras  #使用keras内置的数据集
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data() #加载MNIST数据集
x_train, x_test = x_train / 255.0, x_test / 255.0
#网络设计
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

当我们的网络通过以上设计之后，在 keras 中提供了 compile()和 fit()函数方便实现上述逻辑。首先通过compile 函数指定网络使用的优化器对象，损失函数，评价指标等，通过compile 函数指定网络使用的优化器对象，损失函数，评价指标等。

In [22]:
model.compile(optimizer='adam', #选择adam优化器
              loss='sparse_categorical_crossentropy',#选择sparse_categorical_crossentrop损失函数
              metrics=['accuracy']) #选择sparse_categorical_accuracy评测指标

## 4.2 模型训练

模型设计完成完之后，即可通过 fit()函数送入待训练的数据和验证用的数据集:

In [24]:
a=model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_test, y_test), validation_freq=1)

Epoch 1/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.4992 - accuracy: 0.8484 - val_loss: 0.1190 - val_accuracy: 0.9622
Epoch 2/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1264 - accuracy: 0.9624 - val_loss: 0.0898 - val_accuracy: 0.9719
Epoch 3/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0901 - accuracy: 0.9733 - val_loss: 0.0847 - val_accuracy: 0.9738
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0674 - accuracy: 0.9792 - val_loss: 0.0776 - val_accuracy: 0.9745
Epoch 5/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0537 - accuracy: 0.9819 - val_loss: 0.0839 - val_accuracy: 0.9767


     其中x_train， y_train为训练的数据集 ;epochs 指定训练迭代的epochs 数，这里是5次;validation_data 指定用于验证(测试)的数据集和验证的频率validation_freq。运行上述代码即可实现网络的训练与验证的功能，fit 函数会返回训练过程的数据记录history，其中 history.history 为字典对象，包含了训练过程中的 loss，测量指标等记录项。

In [25]:
a.history # 打印训练记录

{'loss': [0.2825777232646942,
  0.11953180283308029,
  0.08906866610050201,
  0.07019324600696564,
  0.05673723295331001],
 'accuracy': [0.9158833622932434,
  0.9641666412353516,
  0.9733166694641113,
  0.9785333275794983,
  0.9816166758537292],
 'val_loss': [0.11904031038284302,
  0.08979415148496628,
  0.0847097635269165,
  0.07759030908346176,
  0.0839291661977768],
 'val_accuracy': [0.9621999859809875,
  0.9718999862670898,
  0.973800003528595,
  0.9745000004768372,
  0.9767000079154968]}

可以看到会有训练的准确率 ，训练误差，验证准确率，验证误差的记录，可以看到这种操作缩短了开发时间，代码简洁，高效。

## 4.3 模型测试

Model 基类除了可以便捷地完成网络的设计与训练、验证，还可以非常方便的预测和 测试。

In [26]:
test_scores = model.evaluate(x_test, y_test)
print('test loss:', test_scores[0])
print('test acc:', test_scores[1])

313/313 [==============================] - 0s 759us/step - loss: 0.0839 - accuracy: 0.9767
test loss: 0.0839291661977768
test acc: 0.9767000079154968


## 4.4 模型的保存与加载

      模型训练完成后，我们需要将模型保存到文件系统上，从而方便后续的模型测试与部署工作。实际上，在训练时间隔性地保存模型状态也是非常好的习惯，这一点对于训练大规模的网络尤其重要，一般大规模的网络需要训练很长的时间，一旦训练过程被中断或者发生宕机等意外，之前训练的进度将全部丢失。如果能够间断的保存模型状态到文件系统，即使发生宕机等意外，也可以从最近一次的网络状态文件中恢复，从而避免浪费大量的训练时间。因此模型的保存与加载非常重要。
      在 Keras 中，有两种常用的模型保存与加载方法。

### 4.4.1 张量方式

     网络的状态主要体现在网络的结构以及网络层内部张量参数上，因此在拥有网络结构源文件的条件下，直接保存网络张量参数到文件上是最轻量级的一种方式。我们以MNIST 手写数字图片识别模型为例，通过调用Model.save_weights(path)方法就可以将当前的网络参数保存到path 文件上：

In [27]:
model.save_weights('weights.ckpt')

<img src="https://tianchi-public.oss-cn-hangzhou.aliyuncs.com/public/files/forum/161606483491220831616064831038.png"/>

In [28]:
# 重新创建相同的网络结构
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', #选择adam优化器
              loss='sparse_categorical_crossentropy',#选择sparse_categorical_crossentrop损失函数
              metrics=['accuracy']) #选择sparse_categorical_accuracy评测指标
# 从参数文件中读取数据并写入当前网络
model.load_weights('weights.ckpt')

       这种保存与加载网络的方式最为轻量级，文件中保存的仅仅是参数张量的数值，并没有其
      他额外的结构参数。但是它需要使用相同的网络结构才能够恢复网络状态，因此一般在拥
    有网络源文件的情况下使用。

### 4.4.2 网络方式

接下来介绍一种不需要网络源文件，仅仅需要模型参数文件即可恢复出网络模型的方式。通过Model.save(path)函数可以将模型的结构以及模型的参数保存到一个path 文件上，在不需要网络源文件的条件下，通过keras.models.load_model(path)即可恢复网络结构和网络参数。

In [29]:
model.save('model.h5')

<img src="https://tianchi-public.oss-cn-hangzhou.aliyuncs.com/public/files/forum/161606489710927161616064896295.png"/>

此时通过model.h5 文件即可恢复出网络的结构和状态：

In [30]:
net = tf.keras.models.load_model('model.h5')
net.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 128)               100480    
_________________________________________________________________
dense_20 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_21 (Dense)             (None, 10)                650       
Total params: 109,386
Trainable params: 109,386
Non-trainable params: 0
_________________________________________________________________


可以看到，model.h5 文件除了保存了模型参数外，还保存了网络结构信息，不需要提前创 建模型即可直接从文件
恢复出网络 net对象。

# 四、课后作业

     1 知道keras有哪些模块？
     2 掌握每个模块怎样用代码实现，知道keras里面有什么结构。